# Deep Learning Workflow

## Requirements
* ArcGIS Pro v3.4
* Deep Learning Libraries installed
* NAIP Imagery tile to train model
* Labeled drainage features
* NAIP Imagery tile to apply the model

## Workflow
* Export training data for deep learning
* Train deep learning model on primary NAIP image
* Apply deep learning model on secondary NAIP image

In [ ]:
# Packages
from pathlib import Path
import requests
import arcpy

In [ ]:
#Paths

# Raw data folder
raw_data_folder = Path.cwd().parent / 'data' / 'raw'

# NAIP folder
naip_folder = Path.cwd().parent / 'data' / 'raw' / 'naip'

# Training data folder
training_data_folder = Path.cwd().parent / 'data' / '01_training'
modeling_data_folder = Path.cwd().parent / 'data' / '02_modeling'
generated_data_folder = Path.cwd().parent / 'data' / '03_generated'


#### Code for downloading NAIP tiles

In [ ]:
# NAIP download url
naip_url = 'https://dit-cgia-gis-data.s3.amazonaws.com/orthoimagery-program/tiles/2024/sids/OF6i0_37_000_{}_20240203_0219R0.sid'

# NAIP tiles
naip_tiles_ids = [
    '20732901',
    '20731902',
    '20742003',
]

# Fetch all tiles
for the_tile_id in naip_tiles_ids:
    print(f'Fetching tile {the_tile_id}')

    #Create file names
    the_sid_file = str(naip_folder / the_tile_id) + '.sid'
    the_sdw_file = str(naip_folder / the_tile_id) + '.sdw'

    #Check if the file exists
    if Path(the_sid_file).exists():
        print(f' File {the_sid_file} already exists.')
        fetch_sid = False
    else: 
        fetch_sid = True

    if Path(the_sdw_file).exists():
        print(f' File {the_sdw_file} already exists.')
        fetch_sdw = False
    else:
        fetch_sdw = True

    # Construct the data urls
    sid_url = naip_url.format(the_tile_id)
    sdw_url = sid_url.replace('.sid', '.sdw')

    # Fetch the sid file
    if fetch_sid:
        response_sid = requests.get(sid_url)
        if response_sid.status_code == 200:
            binary_data = response_sid.content
            # Process binary_data (e.g., save to file)
            with open(the_sid_file, 'wb') as file:
                file.write(binary_data)
            print(f"  Downloaded {the_sid_file} successfully.")
        else:
            print(f"  Failed to retrieve data: Status code {response.status_code}")
            print(sid_url)

    # Fetch the sdw files
    if fetch_sdw:
        response_sdw = requests.get(sdw_url)
        if response_sdw.status_code == 200:
            binary_data = response_sdw.content
            # Process binary_data (e.g., save to file)
            with open(the_sdw_file, 'wb') as file:
                file.write(binary_data)
            print(f"  Downloaded {the_sdw_file} successfully.")
        else:
            print(f"  Failed to retrieve data: Status code {response.status_code}")
            print(sid_url)

### Code for generating training data
* Requires labeled features & image raster

In [ ]:
#Labeled features
labeled_features = str(Path().cwd().parent / 'data' / 'raw' / 'labeled_ditches.shp')

# Training output
training_output = training_data_folder / 'training_output'

# Train the model
if not training_output.exists():
    tool_result = arcpy.ia.ExportTrainingDataForDeepLearning(
        in_raster=          str(naip_folder / '20732901.sid'), 
        out_folder =        str(training_output),
        in_class_data =     labeled_features, 
        class_value_field = 'Classvalue',
        image_chip_format = 'TIFF', 
        tile_size_x =       512, 
        tile_size_y =       512,
        stride_x =          256,
        stride_y =          256,
        output_nofeature_tiles = False,
        metadata_format =   "RCNN_Masks",  # <----
        #buffer_radius,
        #in_mask_polygons,
        #rotation_angle,
        #reference_system,
        #processing_mode,
        #blacken_around_feature,
        #crop_mode,
        #in_raster2,
        #in_instance_data,
        #instance_class_value_field,
        #min_polygon_overlap_ratio
        )
    print(tool_result.getMessages())

### Training the Model
https://pro.arcgis.com/en/pro-app/latest/tool-reference/image-analyst/train-deep-learning-model.htm

In [ ]:
#Set output modeling folder
modeling_output = modeling_data_folder / 'modeling_output' 

In [ ]:
tool_result = arcpy.ia.TrainDeepLearningModel(
    in_folder = str(training_output),
    out_folder = str(modeling_output),
    max_epochs = 20,
    model_type = 'MASKRCNN',
    batch_size = 1
    )
    
print(tool_result.getMessages())

In [ ]:
arcpy.getMessages()